# Metadata:
* @Autor:Rafael Esteban Torres Herreño
* @Date: 30/04/2022
* @Description: This python notebook shows the use of ML for feelings analysis 
 

#Machine Learning classic (LogisticRegression)

In [23]:
import matplotlib.pyplot as plt
%matplotlib inline

##P0.descargando librerias

---
*   https://stackabuse.com/python-for-nlp-introduction-to-the-pattern-library/

In [24]:
!pip install pyprind

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
import nltk
nltk.download('stopwords') #Descargamos la lista de StopWord (pronombres, artículos, condicionales, demostrativos, adverbios de tiempo y lugar)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
!pip install pattern  #libreria para PLN (Tokenización, derivación, etiquetado de POS, análisis de sentimientos, etc)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##P1.Importando librerias

###P1.1 Genericas

In [27]:
import urllib.request
import numpy as np
import pandas as pd 
import os
import time
import sys
from zipfile import ZipFile       #para descomprimir ZIP
#-----librerias para trabajar PLN
import nltk
#from pattern.es import lemma
from nltk import word_tokenize
from nltk.corpus import stopwords

#----librerias para normalización de textos
import re
from unicodedata import normalize

###P1.2 Machine Learning

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

#CountVectorizer proporcionado por la biblioteca scikit-learn para vectorizar oraciones.
from sklearn.feature_extraction.text import CountVectorizer

##P2. descargando DataSet

In [29]:
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos transcurrido" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()

url_github_Model='https://raw.githubusercontent.com/imesttebanvm/deeplearning/main/dataset_indirectas_twitter-scraper.csv'
urllib.request.urlretrieve(url_github_Model,
                           'DataSet.csv', 
                           reporthook)

100% | 0 MB | 2.60 MB/s | 0 segundos transcurrido

('DataSet.csv', <http.client.HTTPMessage at 0x7f14e6df3f90>)

##P3.cargar en Dataframe

###P3.1 carga el CSV en Dataframe

In [30]:
filepath_dict = {'indirectas':   '/content/DataSet.csv'}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source  # Add another column filled with the source name
    df_list.append(df)

df = pd.concat(df_list)
df.head(10)

,sentence,label,source
0,"full_text,""reply_count"",""retweet_count"",""favor...",NaN,indirectas
1,En twitter ponemos exactamente lo que nadie no...,NaN,indirectas
2,"RT @nacaridportal: Está bien soñar, pero no es...",NaN,indirectas
3,"Te lo resumo? sos lo único que quiero.,""26"",""1...",NaN,indirectas
4,"Te quiero mucho pero no te entiendo.,""18"",""231...",NaN,indirectas
5,Pagaría por ver una peli sobre los mejores mom...,NaN,indirectas
6,¿Cómo de bueno/a sos en la cama?\nhttps://t.co...,NaN,indirectas
7,"Tuvimos conexión desde el día uno.,""38"",""2522""...",NaN,indirectas
8,"Arruino mi vida por la siesta.,""9"",""1553"",""630...",NaN,indirectas
9,Claras señales de que esa persona no te ama\nh...,NaN,indirectas


##P4. conociendo la data (BagWord)

In [31]:
print('RowsXcolums-->',df.shape)
print('indirectas-->',sum(df['source'] == 'indirectas'))


RowsXcolums--> (851, 3)
indirectas--> 851


###P4.1 usando CountVectorizer (ejemplo)

---

CountVectorizer proporcionado por la biblioteca scikit-learn  realiza la tokenización que separa las oraciones en un conjunto de tokens ordenado por vocabulario. Además, elimina la puntuación y los caracteres especiales y puede aplicar otro preprocesamiento a cada palabra

In [32]:
#obtener las palabras que aparecen de acuerdo con el Orden del vocabulario
oraciones = ['Yo no odio a nadie, ni siquiera me cae mal la gente que se portó horrible. Hace rato entendí que a veces las acciones de las personas tienen más que ver con ellos mismos que conmigo. Uno vive más tranquilo cuando perdona y sigue adelante.']
vectorizer = CountVectorizer(min_df=0, lowercase=True,stop_words=stopwords.words('spanish'))   
     
vectorizer.fit(oraciones)

CountVectorizer(min_df=0,
                stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...])

In [33]:
vectorizer.get_feature_names_out()

array(['acciones', 'adelante', 'cae', 'conmigo', 'entendí', 'gente',
       'hace', 'horrible', 'mal', 'mismos', 'nadie', 'odio', 'perdona',
       'personas', 'portó', 'rato', 'sigue', 'siquiera', 'tranquilo',
       'veces', 'ver', 'vive'], dtype=object)

In [34]:
vectorizer.vocabulary_

{'acciones': 0,
 'adelante': 1,
 'cae': 2,
 'conmigo': 3,
 'entendí': 4,
 'gente': 5,
 'hace': 6,
 'horrible': 7,
 'mal': 8,
 'mismos': 9,
 'nadie': 10,
 'odio': 11,
 'perdona': 12,
 'personas': 13,
 'portó': 14,
 'rato': 15,
 'sigue': 16,
 'siquiera': 17,
 'tranquilo': 18,
 'veces': 19,
 'ver': 20,
 'vive': 21}

In [35]:
#bolsa de palabras (Cada documento se representa como un vector)
#vectorizer.transform(oraciones).toarray()
array=vectorizer.transform(oraciones)
print(array.toarray())

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


###P4.2 usando CountVectorizer


In [37]:
df_indirectas = df[df['source'] == 'indirectas']
sentences = df_indirectas['sentence'].values
y = df_indirectas['label'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(
   sentences, y, test_size=0.25, random_state=1000)

###P4.3 generar BagWords 

---

In [38]:
#generamos los vectores de palabras (750x1681) registrosXtamaño del vocabulario
#vectorizer = CountVectorizer(min_df=0, lowercase=True,stop_words=stopwords.words('spanish'))   
vectorizer = CountVectorizer(min_df=0, lowercase=True)   
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)
#print(X_train.toarray())

In [39]:
print(X_train.toarray())
X_train.shape

[[0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


(638, 3765)

##P5.modelo de ML (regresión logistica).

In [40]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Precisión:", score)

ValueError: ignored

###P5.1 evaluando modelo por cada set de datos

In [41]:
for source in df['source'].unique():
    df_source = df[df['source'] == source]
    sentences = df_source['sentence'].values
    y = df_source['label'].values
    sentences_train, sentences_test, y_train, y_test = train_test_split(
        sentences, y, test_size=0.25, random_state=1000)
    vectorizer = CountVectorizer(min_df=0, lowercase=True) 
    vectorizer.fit(sentences_train)
    X_train = vectorizer.transform(sentences_train)
    X_test  = vectorizer.transform(sentences_test)
    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    score = classifier.score(X_test, y_test)
    print('Precisión para datos {}: {:.4f}'.format(source, score))

ValueError: ignored

###P5.2 evaluando modelo con fulldata

In [42]:
sentences = df['sentence'].values
y = df['label'].values
sentences_train, sentences_test, y_train, y_test = train_test_split(
   sentences, y, test_size=0.25, random_state=1000)

vectorizer = CountVectorizer(min_df=0, lowercase=True)   
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)
modeloPLNRegLog = LogisticRegression()
modeloPLNRegLog.fit(X_train, y_train)
score = modeloPLNRegLog.score(X_test, y_test)

print("Precisión:", score)

ValueError: ignored

In [43]:
print(X_train.toarray())
X_train.shape

[[0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


(638, 3765)

###P5.3 Salvando el modelo (serializando)

---
*   salvaremos el modelo y el vocabulario generado del bagWords
*   para ver metodos de exportación:
https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

In [44]:
import pickle
#serializando el modelo
pickle.dump(modeloPLNRegLog, open( 'LogRegression_PLN_classSentimientos_model.sav', 'wb'))
#serializando el vocabulario
feature_list = vectorizer.get_feature_names_out()
pickle.dump(feature_list, open('vocabulary.pkl', 'wb'))

###P5.4 cargando el modelo (serializado)

In [45]:
#cargando el modelo
modeloPLNRegLogNew = pickle.load(open('LogRegression_PLN_classSentimientos_model.sav', 'rb'))
#cargando el vocabulario
vocabularyNew= pickle.load(open('vocabulary.pkl', 'rb'))
vectorizerNew = CountVectorizer(min_df=0, lowercase=True,vocabulary = vocabularyNew)  #creando un nuevo vectorizador con el vocabulario cargado
result = modeloPLNRegLogNew.score(X_test, y_test)   #probado con el test de entrenamiento a ver si funciona
print('Inferencia:%',result)

NotFittedError: ignored

###P5.5 Haciendo predicciones

In [46]:
oracion = ['Yo no odio a nadie, ni siquiera me cae mal la gente que se portó horrible. Hace rato entendí que a veces las acciones de las personas tienen más que ver con ellos mismos que conmigo. Uno vive más tranquilo cuando perdona y sigue adelante.']
X_prueba = vectorizerNew.transform(oracion)  #convertimos la oración al array del bagword

#print('vector:',X_prueba[0].toarray())
predict=modeloPLNRegLogNew.predict(X_prueba)
print(oracion,', El sentimiento es :',predict)

NotFittedError: ignored